In [1]:
2+2

4

In [2]:
import pickle
import pandas as pd
import numpy as np
import os
from tqdm import notebook
from tqdm import tqdm as tq

pd.set_option('display.max_colwidth', 100)

shingling_parameter = 4
spark_similarity_thresh = 0.85
numHashTables = 1

In [3]:
import json

with open('publication.txt', 'r') as f:
    d = json.loads(f.read())

data = []
for key in d.keys():
    data.append({'Publication Venue': key, 'Authors': d[key]})

publication_authors = pd.DataFrame(data)
publication_authors.sample(10, random_state=56).sample(5)

,Publication Venue,Authors
54777,On positive harris recurrence of multiclass queueing networks: a unified approach via fluid limi...,[J G Dai]
66246,Spatially invariant unsupervised object detection with convolutional neural networks,"[E Crawford, J Pineau]"
115321,Wecon Automatic Control Scheme of alarm system in Food Refrigeration Room,[]
61257,ACM Transactions on Applied Perception (TAP),"[Oleg Komogortsev, Corey Holland, Sampath Jayarathna, Alex Karpov, P J Phillips, F Jiang, A Narv..."
129064,2010 IEEE International Conference on Methods and Models in Automation and Robotics,"[D Schindele, H Aschemann]"


In [6]:
lst = list(publication_authors['Publication Venue'])

In [7]:
"""
Experiments with shingling parameters.

Note: Option ‘char_wb’ creates character n-grams only from text inside word boundaries; n-grams at the edges of words are padded with space.
"""

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import time

vectorizer = CountVectorizer(
    ngram_range=(shingling_parameter, shingling_parameter),
    analyzer='char',                           # {‘word’, ‘char’, ‘char_wb’}
    max_features=None
)

start_time = time.time()
shingles = vectorizer.fit_transform(lst)
end_time = time.time()

try:
    print("Shingle length:", len(vectorizer.get_feature_names_out()))
except:
    print("Shingle length:", vectorizer.get_params()['n_features'])

print("Number of shingles", shingles.shape[0])
print()
print(f"Time taken: {end_time - start_time: 0.3f} seconds")

vectorizer.get_feature_names_out()

Shingle length: 154244
Number of shingles 138575

Time taken:  5.615 seconds


array([' ! h', ' ! m', ' ! t', ..., '™åˆ¶', '™ð‘¦', '™‚ä»'], dtype=object)

In [8]:
import sys
import os

os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

from pyspark.ml.feature import MinHashLSH
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.sql import SparkSession


spark = SparkSession\
.builder\
.master("local[28]")\
.config("spark.executor.memory", "21g")\
.config("spark.driver.memory", "21g")\
.config("spark.memory.offHeap.enabled", True)\
.config("spark.memory.offHeap.size","21g")\
.config("spark.driver.maxResultSize", "21g")\
.config("spark.python.worker.memory", "21g")\
.config("spark.scheduler.listenerbus.eventqueue.size", "800000")\
.appName("lsh")\
.getOrCreate()

23/04/25 19:21:26 WARN Utils: Your hostname, archcryptcraft resolves to a loopback address: 127.0.0.1; using 192.168.46.108 instead (on interface wlan0)
23/04/25 19:21:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/04/25 19:21:27 WARN SparkConf: The configuration key 'spark.scheduler.listenerbus.eventqueue.size' has been deprecated as of Spark 2.3 and may be removed in the future. Please use the new key 'spark.scheduler.listenerbus.eventqueue.capacity' instead.
23/04/25 19:21:27 WARN SparkConf: The configuration key 'spark.scheduler.listenerbus.eventqueue.size' has been deprecated as of Spark 2.3 and may be removed in the future. Please use the new key 'spark.scheduler.listenerbus.eventqueue.capacity' instead.
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/25 19:21:27 WARN SparkConf: The configuration key 'spark.scheduler.listenerbus.eventqueue.size' has been deprecated

In [9]:
vector_length = len(vectorizer.get_feature_names_out())

In [10]:
from tqdm import notebook
from tqdm.notebook import tqdm as tq

shingles_d2 = []
data_new = []
publications_d2 = []
mapping = {}
j = 0

for i in tq(range(shingles.shape[0])):
    if shingles[i].getnnz() != 0:
        shingles_d2.append(shingles[i])
        publications_d2.append(lst[i])
        mapping[j] = i
        data_new.append(data[i])
        j += 1

print("number of shingles:", len(shingles_d2))

  0%|          | 0/138575 [00:00<?, ?it/s]

number of shingles: 138235


In [11]:
import random

dataA = []

for i in tq(range(len(shingles_d2))):
    dataA.append((i, Vectors.sparse(vector_length, shingles_d2[i].T.nonzero()[0], shingles_d2[i].T[shingles_d2[i].T.nonzero()[0]].T.toarray()[0])))

random.shuffle(dataA)

  0%|          | 0/138235 [00:00<?, ?it/s]

In [12]:
start_time = time.time()
dfA = spark.createDataFrame(dataA[:], ["id", "features"])
end_time = time.time()
print(f"Time taken: {end_time - start_time: 0.3f} seconds")

Time taken:  7.485 seconds


In [13]:
mh = MinHashLSH(inputCol="features", outputCol="hashes", numHashTables=numHashTables, seed=42)

print("Starting fit")
start_time = time.time()
model = mh.fit(dfA)
end_time = time.time()
print(f"Time taken: {end_time - start_time: 0.3f} seconds")

model.transform(dfA).show()

Starting fit


23/04/25 19:23:44 WARN TaskSetManager: Stage 0 contains a task of very large size (2985 KiB). The maximum recommended task size is 1000 KiB.


Time taken:  2.105 seconds


23/04/25 19:23:46 WARN SparkConf: The configuration key 'spark.scheduler.listenerbus.eventqueue.size' has been deprecated as of Spark 2.3 and may be removed in the future. Please use the new key 'spark.scheduler.listenerbus.eventqueue.capacity' instead.
23/04/25 19:23:46 WARN TaskSetManager: Stage 1 contains a task of very large size (2985 KiB). The maximum recommended task size is 1000 KiB.


+------+--------------------+----------------+
|    id|            features|          hashes|
+------+--------------------+----------------+
|  1117|(154244,[4267,446...|   [[1123923.0]]|
| 72726|(154244,[4503,839...| [[1.6425451E7]]|
|135823|(154244,[4586,469...| [[4.2242788E7]]|
| 21247|(154244,[5339,534...|    [[686993.0]]|
| 87535|(154244,[6451,813...|[[1.01531351E8]]|
| 61764|(154244,[7386,857...| [[2.0712422E7]]|
| 24404|(154244,[5613,106...|   [[2989029.0]]|
| 98395|(154244,[51111,57...|[[4.43530399E8]]|
| 40469|(154244,[80227,89...|[[3.56922687E8]]|
|105383|(154244,[3182,497...|   [[2989029.0]]|
|  7015|(154244,[115442],...|[[2.01618553E8]]|
|  2461|(154244,[5759,609...| [[2.0226174E7]]|
|106284|(154244,[4943,533...|   [[2989029.0]]|
| 70118|(154244,[5166,566...|   [[1123923.0]]|
| 89964|(154244,[9217,965...|[[1.71858789E8]]|
| 62602|(154244,[8867,886...| [[5.4934554E7]]|
|123913|(154244,[6474,805...|   [[1123923.0]]|
| 51358|(154244,[5170,238...|[[2.01618553E8]]|
|116639|(1542

In [14]:
results = model.approxSimilarityJoin(dfA, dfA, 1 - spark_similarity_thresh, distCol="distance")\
    .select(col("datasetA.id").alias("idA"),
            col("datasetB.id").alias("idB"),
            col("distance"))

In [15]:
start_time = time.time()
pd_results = results.toPandas()
end_time = time.time()

print(f"Time taken: {end_time - start_time: 0.3f} seconds")

23/04/25 19:23:52 WARN TaskSetManager: Stage 2 contains a task of very large size (2985 KiB). The maximum recommended task size is 1000 KiB.
23/04/25 19:23:56 WARN TaskSetManager: Stage 3 contains a task of very large size (2985 KiB). The maximum recommended task size is 1000 KiB.


Time taken:  661.656 seconds


In [16]:
pd_results

,idA,idB,distance
0,135450,135450,0.0
1,33358,33358,0.0
2,131299,131299,0.0
3,124803,124803,0.0
4,32570,32570,0.0
...,...,...,...
261800,125048,125048,0.0
261801,1644,1644,0.0
261802,71417,5467,0.0
261803,45875,45875,0.0


In [17]:
results_w_titles = []

for _, row in tq(pd_results.iterrows(), total=len(pd_results)):
    i, j, dist = row['idA'], row['idB'], row['distance']
    i, j = int(i), int(j)

    results_w_titles.append({
        'i': i,
        'j': j,
        'similarity': 1-dist,
        'i_venue': publications_d2[i],
        'j_venue': publications_d2[j],
        'i_authors': data_new[i]['Authors'],
        'j_authors': data_new[j]['Authors'],
        'i_venue_check': data_new[i]['Publication Venue'],
        'j_venue_check': data_new[j]['Publication Venue']
    })

results_df = pd.DataFrame(results_w_titles)
results_df[results_df['i'] < results_df['j']]

  0%|          | 0/261805 [00:00<?, ?it/s]

,i,j,similarity,i_venue,j_venue,i_authors,j_authors,i_venue_check,j_venue_check
7,125576,137324,1.000000,Automatic detection of generated text is easiest when humans are fooled,Automatic Detection of Generated Text is Easiest when Humans are Fooled,"[Daphne Ippolito, Daniel Duckworth, Chris Callison-Burch, Douglas Eck]","[Daphne Ippolito, Daniel Duckworth, Chris Callison-Burch, Douglas Eck]",Automatic detection of generated text is easiest when humans are fooled,Automatic Detection of Generated Text is Easiest when Humans are Fooled
9,7788,40625,1.000000,The augmented lagrange multiplier method for exact recovery of corrupted low-rank matrices,The augmented Lagrange multiplier method for exact recovery of corrupted low-rank matrices,"[Z Lin, M Chen, L Wu, Y Ma, Z Lin, M Chen, Y Ma, Zhouchen Lin, Minming Chen, Yi Ma]","[M Lichman, Z Lin, M Chen, Y Ma, Z Lin, M Chen, L Wu, Y Ma]",The augmented lagrange multiplier method for exact recovery of corrupted low-rank matrices,The augmented Lagrange multiplier method for exact recovery of corrupted low-rank matrices
17,125451,127074,1.000000,ProxSkip: Yes! local gradient steps provably lead to communication acceleration! finally!,ProxSkip: Yes! Local gradient steps provably lead to communication acceleration! Finally!,"[Konstantin Mishchenko, Grigory Malinovsky, Sebastian Stich, Peter RichtÃ¡rik]","[K Mishchenko, G Malinovsky, S Stich, P RichtÃ¡rik]",ProxSkip: Yes! local gradient steps provably lead to communication acceleration! finally!,ProxSkip: Yes! Local gradient steps provably lead to communication acceleration! Finally!
27,31449,40424,0.888889,Journal on Selected Areas in Communications,IEEE journal on selected areas in communications,"[H S Ghadikolaei, F Boccardi, C Fischione, G Fodor, M Zorzi, D Guo, X Wang, Jiayue He, Mung Ma'a...","[W Shi, N Li, X Xie, C-C Chu, R Gadh, J R Barry, J M Kahn, W J Krause, E A Lee, D G Messerschmit...",Journal on Selected Areas in Communications,IEEE journal on selected areas in communications
28,5805,49020,1.000000,IEEE Journal on selected areas in communications,IEEE Journal on selected areas in Communications,"[A Sabelfeld, A C Myers, J G Andrews, S Buzzi, W Choi, S V Hanly, A Lozano, A C Soong, J C Zhang...","[Q Ni, C C Zarakovitis]",IEEE Journal on selected areas in communications,IEEE Journal on selected areas in Communications
...,...,...,...,...,...,...,...,...,...
261773,54110,113739,0.898734,Proceedings of the SIGCHI conference on Human Factors in Computing Systems,Proceedings of the SIGCHI Conference on Human Factors in Computing Systems -CHI '93,"[Clifford Nass, Min Kwan, Lee, Sean Andrist, Tomislav Pejsa, Bilge Mutlu, Michael Gleicher, D S...","[Bonnie A Nardi, Heinrich Schwarz, Allan Kuchinsky, Robert Leichner, Steve Whittaker, Robert Scl...",Proceedings of the SIGCHI conference on Human Factors in Computing Systems,Proceedings of the SIGCHI Conference on Human Factors in Computing Systems -CHI '93
261780,57160,61129,0.882353,Cifar-10 (canadian institute for advanced research,CIFAR-100 (canadian institute for advanced research),"[A Krizhevsky, V Nair, G Hinton, Alex Krizhevsky, Vinod Nair, Geoffrey Hinton]","[Alex Krizhevsky, Vinod Nair, Geoffrey Hinton]",Cifar-10 (canadian institute for advanced research,CIFAR-100 (canadian institute for advanced research)
261787,6152,7906,0.864407,Proceedings of the 2005 ACM symposium on Applied computing,Proceedings of the 2009 ACM symposium on Applied Computing,"[L Reeve, H Han]",[H Park],Proceedings of the 2005 ACM symposium on Applied computing,Proceedings of the 2009 ACM symposium on Applied Computing
261791,22534,34268,0.902439,Proceedings of the IEEE International Symposium on Cluster Computing and the Grid,Proceedings of the 6th IEEE International Symposium on Cluster Computing and the Grid,"[C L Dumitrescu, I Foster]","[A Poshtkuhi, A Abutalebi, L Ayough, S Hessabi]",Proceedings of the IEEE International Symposium on Cluster Computing and the Grid,Proceedings of the 6th IEEE International 

In [18]:
import pickle

with open(f"results_{shingling_parameter}_{spark_similarity_thresh}_{numHashTables}.pkl", 'wb') as file:
    pickle.dump(results_df, file, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
import pickle

with open(f"results_{shingling_parameter}_{spark_similarity_thresh}_{numHashTables}.pkl", 'rb') as file:
    results_df = pickle.load(file)

In [44]:
def find_connected_components(dataframe):
    components = []
    adjacencyList = {}

    for i, j in zip(dataframe['i'], dataframe['j']):
        if i not in adjacencyList:
            adjacencyList[i] = list()
        
        if j not in adjacencyList:
            adjacencyList[j] = list()

        adjacencyList[i].append(j)
        adjacencyList[j].append(i)

    visited = set()
    explored = set()

    def returnComponents(i):
        if i in visited:  # Only count once
            return []

        visited.add(i)
        members = [i]

        for neighbour in adjacencyList.get(i, []):
            members.extend(returnComponents(neighbour))

        explored.add(i)

        return members

    for node in adjacencyList.keys():
        members = returnComponents(node)
        if len(members) > 1:
            components.append(members)
    
    return components

In [45]:
components = find_connected_components(results_df)

In [43]:
len(components)

114990

In [38]:
138575 - len(components)

127954

In [51]:
from tqdm.notebook import tqdm

data_new = []

for component in tqdm(components):
    base_venue = publication_authors.iloc[component[0]]['Publication Venue']
    authors_merged = []
    
    for i in component:
        authors = publication_authors.iloc[i]['Authors']
        authors_merged.extend(authors)
        
    data_new.append({'Publication Venue': base_venue, 'Authors': authors_merged, 'Authors_N': len(authors_merged)})

df_final = pd.DataFrame(data_new)
df_final

  0%|          | 0/10621 [00:00<?, ?it/s]

,Publication Venue,Authors,Authors_N
0,J Numer Math,"[D Arndt, Reza Shokri, Marco Stronati, Congzheng Song, Vitaly Shmatikov]",5
1,CugLM Model,"[H G Lee, S M Yoo, W M Farmer, Rajeev Alur, Thomas A Henzinger]",5
2,Accounting for unobserved confounding in domain generalization,"[A Bellot, M Van Der Schaar, T Sun, M Segu, J Postels, Y Wang, L Van Gool, B Schiele, F Tombari,...",10
3,International Journal of Project Management,"[C Berggren, J SÃ¶derlund, J Thomas, T Mengel, E Osipova, P E Eriksson, M Loosemore, N Andonakis...",26
4,Bayesian recurrent neural networks,"[M Fortunato, C Blundell, O Vinyals, X Yin, W Tan, C Liu]",6
...,...,...,...
10616,International Journal on Document Analysis and Recognition (IJDAR'98),"[Jonathan J Hull, Sergey Mechtaev, Jooyong Yi, Abhik Roychoudhury, S Mechtaev, J Yi, A Roychoudh...",7
10617,IEEE transactions on evolutionary computation,"[E Zitzler, L Thiele, M Laumanns, C M Fonseca, Grunert Da Fonseca, V, K Deb, A Pratap, S Agarwal...",56
10618,Math. Model,"[S Mochen, G Csurka, C Dance, L Fan, J Willamowski, C ; Bray, R Datta, Jz ; Wang, R Mir Datta, D...",31
10619,Proceedings of the 29th ACM International Conference on Information and Knowledge Management (CI...,"[Oliver Benedek Rozemberczki, Rik Kiss, Sarkar, Benedek Rozemberczki, Rik Sarkar, Yana Momchilo...",9


In [55]:
df_final.sort_values(by='Authors_N', ascending=False).head(20)

,Publication Venue,Authors,Authors_N
10171,IEEE Conferences on Networking and Media Convergence,"[T F Ghanem, W S Elkiliani, M Hadhoud, R Rifkin, A Klatau, Peter Auer, Ron Begleiter, Ran El-Yan...",4932
692,Unique in the crowd: The privacy bounds of human mobility. Scientific reports,"[Y.-A De Montjoye, C A Hidalgo, M Verleysen, V D Blondel, D SchlichthÃ¤rle, A Spena, G D'angioli...",4275
5761,IEEE Access,"[S Parhizi, H Lotfi, A Khodaei, S Bahramirad, A Arab, A Khodaei, S K Khator, T Rappaport, S Sun,...",4076
8590,Proceedings of the 3rd International Workshop on Search and Mining User-Generated Contents,"[S Kinsella, V Murdock, N O'hare, Authors, Antoine Bordes, LÃ©on Bottou, Patrick Gallinari, Jas...",3932
3478,Phase transitions in the coloring of random graphs,"[L ZdeborovÃ¡, F Krzakala, I Filippidis, D V Dimarogonas, K J Kyriakopoulos, S G Loizou, K J Kyr...",3684
4743,IEEE Trans. Inf. Theory,"[A F Dana, B Hassibi, T Cover, A El Gamal, G Kramer, M Gastpar, P Gupta, P Gupta, P R Kumar, M G...",2868
927,IEEE Transactions on Information Theory,"[R J Solomonoff, T M Cover, P Hart, L GyÃ¶rfi, G Morvai, S Yakowitz, A KrzyÅ¼ak, M Pawlak, R Gra...",2725
4886,Foundation and Trends in Machine Learning,"[S Bubeck, N Cesa-Bianchi, Thomas Erneux, Julien Javaloyes, Matthias Wolfrum, Serhiy Yanchuk, A ...",2540
279,Workshop on Algorithmic Foundations of Robotics,"[K Leung, N ArÃ©chiga, M Pavone, M Alviano, F Calimeri, C Dodaro, D FuscÃ , N Leone, S Perri, F ...",2332
5667,Uniswap v3 core,"[Hayden Adams, Noah Zinsmeister, Moody Salem, River Keefer, Dan Robinson, V Dignum, Ravi Ganti, ...",2328


In [56]:
import pickle

with open("results_components.pkl", 'wb') as file:
    pickle.dump(df_final, file, protocol=pickle.HIGHEST_PROTOCOL)